### Part 4

In [101]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline

In [102]:
data = loadmat("data/ex3data1.mat")

In [103]:
data

{'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ..., 
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [104]:
data['X'].shape, data['y'].shape

((5000, 400), (5000, 1))

In [105]:
def sigmoid(z):  
    return 1 / (1 + np.exp(-z))

In [106]:
def cost(theta, X, y, learningRate):  
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = (learningRate / 2 * len(X)) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    return np.sum(first - second) / (len(X)) + reg

In [107]:
def gradient(theta, X, y, learningRate):  
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    parameters = int(theta.ravel().shape[1])
    error = sigmoid(X * theta.T) - y

    grad = ((X.T * error) / len(X)).T + ((learningRate / len(X)) * theta)

    # intercept gradient is not regularized
    grad[0, 0] = np.sum(np.multiply(error, X[:,0])) / len(X)

    return np.array(grad).ravel()

In [108]:
from scipy.optimize import minimize

def one_vs_all(X, y, num_labels, learning_rate):  
    rows = X.shape[0]
    params = X.shape[1]

    # k X (n + 1) array for the parameters of each of the k classifiers
    all_theta = np.zeros((num_labels, params + 1))

    # insert a column of ones at the beginning for the intercept term
    X = np.insert(X, 0, values=np.ones(rows), axis=1)

    # labels are 1-indexed instead of 0-indexed
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))

        # minimize the objective function
        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, learning_rate), method='TNC', jac=gradient)
        all_theta[i-1,:] = fmin.x

    return all_theta

In [109]:
rows = data['X'].shape[0]
params = data['X'].shape[1]

In [110]:
rows

5000

In [111]:
all_theta = np.zeros((10, params + 1))

In [112]:
X = np.insert(data['X'], 0, values=np.ones(rows), axis=1)

In [113]:
theta = np.zeros(params + 1)

In [114]:
y_0 = np.array([1 if label == 0 else 0 for label in data['y']])  
y_0 = np.reshape(y_0, (rows, 1))

In [115]:
X.shape, y_0.shape, theta.shape, all_theta.shape

((5000, 401), (5000, 1), (401,), (10, 401))

In [116]:
np.unique(data['y'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [117]:
all_theta = one_vs_all(data['X'], data['y'], 10, 1)
all_theta

array([[ -3.70247932e-05,   0.00000000e+00,   0.00000000e+00, ...,
         -2.24803601e-10,   2.31962907e-11,   0.00000000e+00],
       [ -8.96250748e-05,   0.00000000e+00,   0.00000000e+00, ...,
          7.26120858e-09,  -6.19965325e-10,   0.00000000e+00],
       [ -8.39553375e-05,   0.00000000e+00,   0.00000000e+00, ...,
         -7.61695635e-10,   4.64917650e-11,   0.00000000e+00],
       ..., 
       [ -7.00832718e-05,   0.00000000e+00,   0.00000000e+00, ...,
         -6.92009371e-10,   4.29241678e-11,   0.00000000e+00],
       [ -7.65188092e-05,   0.00000000e+00,   0.00000000e+00, ...,
         -8.09503517e-10,   5.31058853e-11,   0.00000000e+00],
       [ -6.63412494e-05,   0.00000000e+00,   0.00000000e+00, ...,
         -3.49765987e-09,   1.13668573e-10,   0.00000000e+00]])

In [118]:
def predict_all(X, all_theta):  
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]

    # same as before, insert ones to match the shape
    X = np.insert(X, 0, values=np.ones(rows), axis=1)

    # convert to matrices
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)

    # compute the class probability for each class on each training instance
    h = sigmoid(X * all_theta.T)

    # create array of the index with the maximum probability
    h_argmax = np.argmax(h, axis=1)

    # because our array was zero-indexed we need to add one for the true label prediction
    h_argmax = h_argmax + 1

    return h_argmax

In [119]:
y_pred = predict_all(data['X'], all_theta)  
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]  
accuracy = (sum(map(int, correct)) / float(len(correct)))  
print('accuracy = {0}%'.format(accuracy * 100))

accuracy = 74.6%


Better in online tutorial...and better with python 2.7

In [ ]:
# Testing sklearn

In [55]:
from sklearn import linear_model

In [57]:
logistic_classifier = linear_model.LogisticRegression(C=100.0)

In [59]:
logistic_classifier.fit(data['X'], data['y'])

C:\Program Files\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=100.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [61]:
logistic_classifier.score(data['X'], data['y'])

0.97440000000000004